In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from bokeh.plotting import figure, output_file, show, output_notebook
from bokeh.models import Span
from shapely.geometry import Point
import geopandas as gpd
import glob
from datetime import datetime
from bokeh.layouts import Row, column, gridplot
from bokeh.models import Title, Legend, TapTool, Range1d
import matplotlib as mpl
output_notebook()


Loading BokehJS ...

In [2]:
from shapely.geometry import Point
import geopandas as gpd

def convert_to_gdf(df):
    geometry = [Point(xy) for xy in zip(df.client_longitude, df.client_latitude)]
    gdf = gpd.GeoDataFrame(df, crs="EPSG:4326", geometry=geometry)

    return gdf

c:\Users\sahit\anaconda3\envs\datapartnership\lib\site-packages\geopandas\_compat.py:111: UserWarning: The Shapely GEOS version (3.10.1-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.9.1-CAPI-1.14.2). Conversions between both will be slow.
  warnings.warn(


In [10]:
color_palette = [  '#4E79A7',  # Blue
    '#F28E2B',  # Orange
    '#E15759',  # Red
    '#76B7B2',  # Teal
    '#59A14F',  # Green
    '#EDC948',  # Yellow
    '#B07AA1',  # Purple
    '#FF9DA7',  # Pink
    '#9C755F',  # Brown
    '#BAB0AC',  # Gray
    '#7C7C7C',  # Dark gray
    '#6B4C9A',  # Violet
    '#D55E00',  # Orange-red
    '#CC61B0',  # Magenta
    '#0072B2',  # Bright blue
    '#329262',  # Peacock green
    '#9E5B5A',  # Brick red
    '#636363',  # Medium gray
    '#CD9C00',  # Gold
    '#5D69B1',  # Medium blue
]

# Ookla's Number of Speedtest Intelligence Users

In [3]:
ios = pd.concat([pd.read_csv('../data/ookla/final/ios_counts_turkey.csv'), pd.read_csv('../data/ookla/final/ios_counts_turkey_02_28.csv'), pd.read_csv('../data/ookla/final/ios_counts_turkey_03_07.csv')])
stnet = pd.concat([pd.read_csv('../data/ookla/final/stnet_counts_turkey.csv'), pd.read_csv('../data/ookla/final/stnet_counts_turkey_02_28.csv'), pd.read_csv('../data/ookla/final/stnet_counts_turkey_03_07.csv')])
stdesktop = pd.concat([pd.read_csv('../data/ookla/final/stdesktop_counts_turkey.csv'), pd.read_csv('../data/ookla/final/stdesktop_counts_turkey_02_28.csv'), pd.read_csv('../data/ookla/final/stdesktop_counts_turkey_03_07.csv')])
android = pd.concat([pd.read_csv('../data/ookla/final/stdesktop_counts_turkey.csv'), pd.read_csv('../data/ookla/final/android_counts_turkey_02_28.csv'), pd.read_csv('../data/ookla/final/android_counts_turkey_03_07.csv')])

In [4]:
ios.drop(columns='Unnamed: 0', inplace = True)
stnet.drop(columns='Unnamed: 0', inplace = True)
stdesktop.drop(columns='Unnamed: 0', inplace = True)
android.drop(columns='Unnamed: 0', inplace = True)

In [196]:
df = pd.concat([ios, stnet, stdesktop, android]).groupby(['adm2_en', 'date']).sum().reset_index()
df = df[(df['date']>'2023-01-03')&(df['date']<'2023-03-13')]

In [197]:
turkey_adm2 = gpd.read_file('../data/turkey_administrativelevels0_1_2/tur_polbna_adm2.shp')
df = df.merge(turkey_adm2[['adm2_en', 'adm1_en']], how = 'right')
df = df.groupby(['adm1_en', 'date']).sum().reset_index()
df['date'] = df['date'].apply(lambda x: pd.to_datetime(x))
#week = df.groupby(['adm1_en','date']).sum().reset_index()

In [198]:
turkey_adm2 = gpd.read_file('../data/turkey_administrativelevels0_1_2/tur_polbna_adm2.shp')
turkey_aoi = gpd.read_file('../data/TUR_tessellation.geojson')
turkey_adm2.rename(columns = {'pcode':'ADM2_PCODE'}, inplace=True)
turkey_aoi = turkey_aoi[['ADM1_PCODE', 'ADM2_PCODE']].merge(turkey_adm2, on = ['ADM2_PCODE'])
affected_adm1 = list(turkey_aoi['adm1_en'].unique())
#adm1 = [x.capitalize() for x in adm1]

In [199]:
#week['week_date'] = df['date'] - pd.to_timedelta(7, unit='d')

# #calculate sum of values, grouped by week
week = df.groupby([pd.Grouper(key='date', freq='W-MON'), 'adm1_en']).mean().reset_index()

week['user_count'] = week['user_count'].apply(lambda x: round(x,0))

In [11]:
for date in ['2023-02-13', '2023-02-20', '2023-02-27', '2023-03-06']:
        
    df = week[week['week_date']==date][['week_date','adm2_en', 'user_count']].merge(turkey_adm2[['adm2_en', 'adm1_en']],on='adm2_en', how = 'right').reset_index(drop=True)
    df['week_date'] = date
    df.sort_values(by='adm2_en').to_csv(f'../data/ookla/final/turkiye_users_{date}.csv')

In [200]:
#title = "Network Connectivity Trend"#"Business Activity Trends per Business Vertical in February 2022 (Compared to Pre-COVID Baseline)"

p2 = figure( x_axis_type = 'datetime', width = 800, height = 400, toolbar_location='above')
p2.add_layout(Legend(), "right")

affected_regions = ['GAZIANTEP', 'HATAY', 'KAHRAMANMARAS']

# for id, adm1 in enumerate(turkey_adm2['adm1_en'].unique()):
#     df1 = df[df['adm1_en']==adm1][['date', 'user_count']].groupby('date').sum().reset_index().sort_values(by='date')
#     if adm1 not in (['GAZIANTEP', 'HATAY', 'KAHRAMANMARAS']):
#         p2.line(df1['date'], df1['user_count'], line_width=2, line_color = '#d2d2d2')

for id, adm1 in enumerate(affected_adm1):
    df1 = df[df['adm1_en']==adm1][['date', 'user_count']].sort_values(by='date')
    p2.line(df1['date'], df1['user_count'], line_width=2, line_color = color_palette[affected_adm1.index(adm1)], legend_label=adm1)

p2.renderers.extend([
        Span(
            location=datetime(2023, 2, 6),
            dimension="height",
            line_color='#7C7C7C',
            line_width=1,
            line_dash=(4,4)
      ),
        Span(
            location=datetime(2023, 2, 20),
            dimension="height",
            line_color='#7C7C7C',
            line_width=1,
            line_dash=(4,4)
        ),
    ]
)

#p2.yaxis.minor_tick_line_color = None
#p2.y_range = Range1d(0, 0.8)
p2.legend.click_policy='hide'


title_fig = figure(title='Changes to daily users using SpeedTest in affected Govervorates', toolbar_location=None,width=1200, height=50, )
title_fig.title.align = "left"
title_fig.title.text_font_size = "20pt"
title_fig.border_fill_alpha = 0
title_fig.outline_line_width=0
tap = TapTool(behavior="inspect")
p2.add_tools(tap)

sub_title = figure(title='Source: Ookla Speedtest Intelligence Data Extracts', toolbar_location=None,width=1000, height=50, )
sub_title.title.align = "left"
sub_title.title.text_font_size = "10pt"
sub_title.title.text_font_style="normal"
sub_title.border_fill_alpha = 0
sub_title.outline_line_width=0


row = Row(children=[p2])
grid = gridplot([[row]])
layout = column(title_fig, grid, sub_title)

show(layout)

In [201]:
p2 = figure(x_axis_type = 'datetime', width = 800, height = 400, toolbar_location='above')
p2.add_layout(Legend(), "right")

affected_regions = ['GAZIANTEP', 'HATAY', 'KAHRAMANMARAS']

# for id, adm1 in enumerate(turkey_adm2['adm1_en'].unique()):
#     df = week[week['adm1_en']==adm1][['week_date', 'user_count']].groupby('week_date').mean().reset_index().sort_values(by='week_date')
#     if adm1 not in (['GAZIANTEP', 'HATAY', 'KAHRAMANMARAS']):
#         p2.line(df['week_date'], df['user_count'], line_width=2, line_color = '#d2d2d2')

for id, adm1 in enumerate(affected_adm1):
    df1 = week[week['adm1_en']==adm1][['date', 'user_count']].groupby('date').mean().reset_index().sort_values(by='date')
    p2.line(df1['date'], df1['user_count'], line_width=2, line_color = color_palette[affected_adm1.index(adm1)], legend_label=adm1)

p2.renderers.extend([
        Span(
            location=datetime(2023, 2, 6),
            dimension="height",
            line_color='#7C7C7C',
            line_width=1,
            line_dash=(4,4)
      ),
        Span(
            location=datetime(2023, 2, 20),
            dimension="height",
            line_color='#7C7C7C',
            line_width=1,
            line_dash=(4,4)
        ),
    ]
)

p2.yaxis.minor_tick_line_color = None
#p2.y_range = Range1d(0, 0.8)
p2.legend.click_policy='hide'


title_fig = figure(title='Changes to weekly average of users using SpeedTest in affected Govervorates', toolbar_location=None,width=1200, height=50, )
title_fig.title.align = "left"
title_fig.title.text_font_size = "20pt"
title_fig.border_fill_alpha = 0
title_fig.outline_line_width=0
tap = TapTool(behavior="inspect")
p2.add_tools(tap)

sub_title = figure(title='Source: Ookla Speedtest Intelligence Data Extracts', toolbar_location=None,width=1000, height=50, )
sub_title.title.align = "left"
sub_title.title.text_font_size = "10pt"
sub_title.title.text_font_style="normal"
sub_title.border_fill_alpha = 0
sub_title.outline_line_width=0


row = Row(children=[p2])
grid = gridplot([[row]])
layout = column(title_fig, grid, sub_title)

show(layout)